# Releasing an assignment

**These instructions are currently incomplete**.

Creating the release version of an assignment consists of creating a
student version of a notebook that has solutions omited, based on an already created teacher, or master, version.

To create the master/teacher notebook that has exercises, solutions and tests, see the documentation on the ["Create Assignment" extension](2 - Developing assignments.ipynb). 

In the `release_example` directory we have an two example master/teacher notebooks:

* [Problem 1](release_example/teacher/Problem 1.ipynb)
* [Problem 2](release_example/teacher/Problem 2.ipynb)

There is also a [header notebook](release_example/header.ipynb) which will be prepended to the teacher notebooks in the release version.

In [1]:
cd release_example

/Users/jhamrick/project/tools/nbgrader/docs/user_guide/release_example


In [2]:
!ls -R

header.ipynb              nbgrader_assign_config.py teacher

./teacher:
Problem 1.ipynb Problem 2.ipynb


Additionally, there is a `nbgrader_assign_config.py` file, which specifies the configuration for `nbgrader`:

In [3]:
!cat nbgrader_assign_config.py

c = get_config()

import os
cwd = os.getcwd()

c.AssignApp.notebooks = ['teacher/*.ipynb']
c.AssignApp.output_dir = os.path.join(cwd, 'student')
c.IncludeHeaderFooter.header = os.path.join(cwd, 'header.ipynb')

# These are only used if run with --save-cells
c.SaveGradeCells.assignment_id = "Problem Set 1"
c.SaveGradeCells.db_url = "sqlite:////tmp/nbgrader_example.db"


This config file says that we should convert all the notebooks in the `teacher` directory, and save them into a new directory called `student`. We additionally specify the header file, which will be prepended to the beginning of the assignment. For other options, please see the [`nbgrader assign` documentation](../command_line_tools/nbgrader assign.ipynb).

Now, to create the release version, all we have to do is:

In [4]:
!nbgrader assign

[AssignApp] Using existing profile dir: u'/Users/jhamrick/.nbgrader/profile_default'


[AssignApp] Changing to directory: teacher
[AssignApp] Converting notebook Problem 1.ipynb to notebook
[AssignApp] Support files will be in Problem 1_files/


[AssignApp] Writing output to directory: student
[AssignApp] Writing 9866 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/release_example/student/Problem 1.ipynb
[AssignApp] Changing to directory: teacher
[AssignApp] Converting notebook Problem 2.ipynb to notebook
[AssignApp] Support files will be in Problem 2_files/


[AssignApp] Writing output to directory: student
[AssignApp] Writing 2689 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/release_example/student/Problem 2.ipynb


We could have also specified our options on the command line, e.g.:

In [5]:
!nbgrader assign --output-dir=../student --header=../header.ipynb teacher/*.ipynb

[AssignApp] Using existing profile dir: u'/Users/jhamrick/.nbgrader/profile_default'
[AssignApp] Changing to directory: teacher


[AssignApp] Converting notebook Problem 1.ipynb to notebook
[AssignApp] Support files will be in Problem 1_files/


[AssignApp] Writing output to directory: student
[AssignApp] Writing 9866 bytes to ../student/Problem 1.ipynb
[AssignApp] Changing to directory: teacher
[AssignApp] Converting notebook Problem 2.ipynb to notebook
[AssignApp] Support files will be in Problem 2_files/
[AssignApp] Writing output to directory: student
[AssignApp] Writing 2689 bytes to ../student/Problem 2.ipynb


These are the resulting student notebooks:

* [Problem 1](release_example/student/Problem 1.ipynb)
* [Problem 2](release_example/student/Problem 2.ipynb)

They are versions of the assignment that can be distributed to the students, who can fill out the relevant locations of the notebook with their solutions, and eventually turn it back in.

## Recording grade cells

If you want to record information about grade cells into the database (for example, to then use those grades and point values, rather than the ones in the notebooks students have returned), then you can optionally run `nbgrader assign` with the `--save-cells` flag.

Before we can run this command, though, we need to actually what the assignment is that we're recording grade cells for. We can do this using the API provided by nbgrader, which provides access to a MongoDB to store information about students and their grades:

In [6]:
# remove an existing database
dbpath = "/tmp/nbgrader_example.db"
if os.path.exists(dbpath):
    os.remove(dbpath)

# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook
gb = Gradebook("sqlite:///" + dbpath)

In [7]:
# add the assignment to the database
gb.add_assignment("Problem Set 1")

Problem Set 1

Now, we can run our `nbgrader` command:

In [8]:
!nbgrader assign --save-cells

[AssignApp] Using existing profile dir: u'/Users/jhamrick/.nbgrader/profile_default'
[AssignApp] Changing to directory: teacher
[AssignApp] Converting notebook Problem 1.ipynb to notebook
[AssignApp] Support files will be in Problem 1_files/


[AssignApp] Writing output to directory: student
[AssignApp] Writing 9866 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/release_example/student/Problem 1.ipynb
[AssignApp] Changing to directory: teacher
[AssignApp] Converting notebook Problem 2.ipynb to notebook
[AssignApp] Support files will be in Problem 2_files/


[AssignApp] Writing output to directory: student
[AssignApp] Writing 2689 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/release_example/student/Problem 2.ipynb


After doing this, we should be able to see the grade cells in the database:

In [9]:
# print out how many 
gb = Gradebook("sqlite:///" + dbpath)
notebook = gb.find_notebook("Problem 1", "Problem Set 1")
print("There are {} grade cells in Problem Set 1, Problem 1".format(len(notebook.grade_cells)))

There are 8 grade cells in Problem Set 1, Problem 1
